In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import cv2

np.set_printoptions(linewidth=np.inf)
np.set_printoptions(edgeitems=30) 

In [2]:

img = cv2.imread('\data\chest_xray\test\NORMAL\IM-0009-0001.jpeg', cv2.IMREAD_GRAYSCALE)

print(img)

plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("Exemple d'image")
plt.show()

# Inspection des métadonnées de l'image
print(f"Shape (dimensions)    : {img.shape}")
print(f"Type des valeurs      : {img.dtype}")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 21-22: malformed \N character escape (326396872.py, line 1)

## Exlploration des donées 

In [ ]:
def explore_images(data_dir, img_read_type: int):
    """
    Explore un répertoire d'images organisé par classe (sous-dossiers).

    - Affiche la première image de chaque sous-dossier.
    - Compte le nombre total d’images.
    - Recueille les tailles uniques des images rencontrées.

    Parameters
    ----------
    data_dir : str
        Chemin du dossier contenant les sous-dossiers d’images.
    img_read_type : int
        Mode de lecture OpenCV (ex. cv2.IMREAD_COLOR ou cv2.IMREAD_GRAYSCALE).
    """
    
    unique_shapes = set()
    displayed = set()
    total_images = 0

    for label_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.isdir(class_dir):
            continue

        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                filepath = os.path.join(class_dir, filename)
                img = cv2.imread(filepath, img_read_type)
                if img is None:
                    continue  # image illisible
                unique_shapes.add(img.shape)
                total_images += 1

                # Afficher la première image de chaque dossier
                if label_name not in displayed:
                    displayed.add(label_name)
                    plt.imshow(img if img_read_type == cv2.IMREAD_GRAYSCALE else cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    plt.title(f"Exemple: {label_name}")
                    plt.axis('off')
                    plt.show()

    print(f"\nTotal d'images dans '{data_dir}' : {total_images}")
    print(f"\nTotal tailles d'images rencontrées : {len(unique_shapes)}")


explore_images("data/chest_ray/train", cv2.IMREAD_COLOR)
explore_images("data/chest_ray/test", cv2.IMREAD_COLOR)

## Chargement des images 

In [ ]:
def load_data(data_dir, img_read_type: int, target_size=(64, 64)):


    X = []
    y = []
    label_map = {'benin': 0, 'malin': 1}

    for label_name in ['benin', 'malin']:
        class_dir = os.path.join(data_dir, label_name)
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                filepath = os.path.join(class_dir, filename)
                img = cv2.imread(filepath, img_read_type)
                if img is None:
                    continue  # image illisible, on passe
                img = cv2.resize(img, target_size)
                X.append(img)
                y.append(label_map[label_name])

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    return X, y


# Chargement des images en niveau de gris
X_train, y_train = load_data('data/chest_ray/train', cv2.IMREAD_GRAYSCALE)
X_test, y_test = load_data('data/chest_ray/test', cv2.IMREAD_GRAYSCALE)

## Preprocessing

In [ ]:

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

X_train /= 255.0
X_test /= 255.0

X_train_reshape = X_train.reshape(X_train.shape[0], 64 * 64)
X_test_reshape = X_test.reshape(X_test.shape[0], 64 * 64)